<a href="https://colab.research.google.com/github/svmmerfantasy/Bangkit-2023/blob/main/29_May_Collaborative_Filtering_%5BFix(%3F)%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Library

In [5]:
# !sudo pip3 install keras

In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.layers import Embedding, Flatten, Input, Concatenate, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import RootMeanSquaredError

#Preparing Dataset


#Cleaning Dataset

In [7]:
mentee = pd.read_csv("Mentees.csv")
mentor = pd.read_csv("Mentors.csv")
rating = pd.read_csv("Ratings.csv")

In [8]:
cols_mente = {"ID MENTEE" : "MenteeID", "SUBJECT" : "Subject" ,"NAME" : "MenteeName", "LOCATION" : "Location"}
mentee.rename(columns=cols_mente, inplace=True)
mentee_df = mentee.loc[0:1499, 'MenteeID' : 'LONGITUDE']

In [9]:
cols_mentor = {"ID MENTOR" :"TutorID",  "SUBJECT" : "Subject", "NAME" : "TutorName", "LOCATION" : "Location", "MATKUL" : "Subjects"}
mentor.rename(columns=cols_mentor, inplace=True)
mentor_df = mentor.loc[0:499, 'TutorID' : 'LONGITUDE']

In [10]:
mentors_df = mentor_df.drop(columns=['Subject'])

In [11]:
cols_rating = {"ID MENTEE" : "MenteeID", "ID MENTOR" : "TutorID", "RATING" : "Rating"}
rating.rename(columns=cols_rating, inplace=True)
rating_df = rating.loc[0:12625, 'MenteeID' : 'Rating']
rating_df['Rating'] = rating_df['Rating'].astype(np.int64)

#Preprocessing Dataset

In [12]:
# tutor_rating.Rating.value_counts()
# tutor_rating = pd.merge(rating_df, mentor_df, on="TutorID")
tutor_rating = rating_df.merge(mentors_df, on="TutorID")

tutor_rating

,MenteeID,TutorID,Rating,TutorName,Subjects,Location,SUBDISTRICT,LATITUDE,LONGITUDE
0,B36652,A16227,3,Andrianto,Ekonomi,Jakarta,JOHAR BARU,-6.183050,106.854320
1,B63077,A16227,5,Andrianto,Ekonomi,Jakarta,JOHAR BARU,-6.183050,106.854320
2,B14364,A16227,5,Andrianto,Ekonomi,Jakarta,JOHAR BARU,-6.183050,106.854320
3,B61521,A16227,4,Andrianto,Ekonomi,Jakarta,JOHAR BARU,-6.183050,106.854320
4,B62409,A16227,5,Andrianto,Ekonomi,Jakarta,JOHAR BARU,-6.183050,106.854320
...,...,...,...,...,...,...,...,...,...
12621,B29822,A17248,3,Junaeri,Matematika,Yogyakarta,PAKUALAMAN,-7.800546,110.375171
12622,B64162,A17248,3,Junaeri,Matematika,Yogyakarta,PAKUALAMAN,-7.800546,110.375171
12623,B32844,A17248,4,Junaeri,Matematika,Yogyakarta,PAKUALAMAN,-7.800546,110.375171
12624,B97893,A17248,5,Junaeri,Matematika,Yogyakarta,PAKUALAMAN,-7.800546,110.375171


In [13]:
rating_count = (tutor_rating.
                groupby(by = ["TutorName"])["Rating"].
                count().
                reset_index().
                rename(columns = {'Rating' : 'RatingCount_Tutor'})
                [['TutorName', 'RatingCount_Tutor']]
                )     

In [14]:
popular_tutor_rating = pd.merge(rating_count, tutor_rating, left_on="TutorName", right_on="TutorName", how='left')

popular_tutor_rating.head(100)

,TutorName,RatingCount_Tutor,MenteeID,TutorID,Rating,Subjects,Location,SUBDISTRICT,LATITUDE,LONGITUDE
0,A. Solikhin,17,B55212,A49119,4,Fisika,Denpasar,DENPASAR BARAT,-8.662372,115.197646
1,A. Solikhin,17,B74603,A49119,3,Fisika,Denpasar,DENPASAR BARAT,-8.662372,115.197646
2,A. Solikhin,17,B76968,A49119,3,Fisika,Denpasar,DENPASAR BARAT,-8.662372,115.197646
3,A. Solikhin,17,B44857,A49119,4,Fisika,Denpasar,DENPASAR BARAT,-8.662372,115.197646
4,A. Solikhin,17,B55497,A49119,4,Fisika,Denpasar,DENPASAR BARAT,-8.662372,115.197646
...,...,...,...,...,...,...,...,...,...,...
95,Abdul Kholik,21,B33409,A15476,4,Bahasa Indonesia,Tangerang,PAKUHAJI,-6.054425,106.605534
96,Abdul Malik,16,B30079,A59067,3,Sosiologi,Yogyakarta,WIROBRAJAN,-7.802891,110.350054
97,Abdul Malik,16,B21929,A59067,5,Sosiologi,Yogyakarta,WIROBRAJAN,-7.802891,110.350054
98,Abdul Malik,16,B33877,A59067,5,Sosiologi,Yogyakarta,WIROBRAJAN,-7.802891,110.350054


#Encode Dataset

In [15]:
# Prepare the required dataframe to feed the model
popular_tutor_rating_df = popular_tutor_rating[['MenteeID', 'TutorID', 'Rating']]
popular_tutor_rating_df

,MenteeID,TutorID,Rating
0,B55212,A49119,4
1,B74603,A49119,3
2,B76968,A49119,3
3,B44857,A49119,4
4,B55497,A49119,4
...,...,...,...
12621,B55161,A57575,4
12622,B86900,A57575,3
12623,B80176,A57575,3
12624,B25637,A57575,4


In [16]:
data_cosine = popular_tutor_rating[['MenteeID', 'TutorID', 'Rating', 'Subjects']]

In [17]:
# Encode user_id and mentor_id using LabelEncoder
mentee_encoder = LabelEncoder()
mentor_encoder = LabelEncoder()
subject_encoder = LabelEncoder()

popular_tutor_rating_df = popular_tutor_rating_df.copy()

# popular_tutor_rating.loc[:, 'Subjects'] = subject_encoder.fit_transform(popular_tutor_rating['Subjects'])
popular_tutor_rating_df.loc[:, 'MenteeID'] = mentee_encoder.fit_transform(popular_tutor_rating_df['MenteeID'])
popular_tutor_rating_df.loc[:, 'TutorID'] = mentor_encoder.fit_transform(popular_tutor_rating_df['TutorID'])

#
data_cosine.loc[:, 'MenteeID'] = mentee_encoder.fit_transform(data_cosine['MenteeID'])
data_cosine.loc[:, 'TutorID'] = mentor_encoder.fit_transform(data_cosine['TutorID'])
data_cosine.loc[:, 'Subjects'] = subject_encoder.fit_transform(data_cosine['Subjects'])

<ipython-input-17-01f684ae0d57>:9: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  popular_tutor_rating_df.loc[:, 'MenteeID'] = mentee_encoder.fit_transform(popular_tutor_rating_df['MenteeID'])
<ipython-input-17-01f684ae0d57>:10: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  popular_tutor_rating_df.loc[:, 'TutorID'] = mentor_encoder.fit_transform(popular_tutor_rating_df['TutorID'])
<ipython-input-17-01f684ae0d57>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

In [18]:
data_cosine

,MenteeID,TutorID,Rating,Subjects
0,747,212,4,4
1,1054,212,3,4
2,1100,212,3,4
3,583,212,4,4
4,751,212,4,4
...,...,...,...,...
12621,745,254,4,2
12622,1279,254,3,2
12623,1156,254,3,2
12624,223,254,4,2


#Based on rating using cosine similarity

In [19]:
rating_matrix = popular_tutor_rating_df.pivot_table(index='MenteeID', columns='TutorID', values='Rating', fill_value=0)
rating_similarity  = cosine_similarity(rating_matrix)
rating_matrix

TutorID,0,1,2,3,4,5,6,7,8,9,...,489,490,491,492,493,494,495,496,497,498
MenteeID,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1490,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1491,0,0,0,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1492,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#Split Dataset

In [23]:
train_data, test_data = train_test_split(data_cosine, test_size=0.2)

print(f"train_data shape : {train_data.shape} \n test_data shape: {test_data.shape}")

train_data shape : (10100, 4) 
 test_data shape: (2526, 4)


#Build ML Model Using TensorFlow

In [24]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(10,)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    # tf.keras.layers.Conv1D(128, 5, activation='relu'),
    # tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(1)
  ])
#Compile model
model.compile(optimizer=Adam(0.0001),
              loss='mean_squared_error',
              metrics=[RootMeanSquaredError()])
# model.build(input_shape=(None,10))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 128)               1408      
                                                                 
 dense_5 (Dense)             (None, 256)               33024     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_6 (Dense)             (None, 256)               65792     
                                                                 
 dense_7 (Dense)             (None, 1)                 257       
                                                                 
Total params: 100,481
Trainable params: 100,481
Non-trainable params: 0
_________________________________________________________________


In [25]:
history = model.fit([train_data['MenteeID'], train_data['TutorID'], train_data['Subjects']], y=train_data['Rating'], epochs=150, batch_size=32)

Epoch 1/150


ValueError: ignored

In [ ]:
# # Define the number of mentees, mentors
# num_mentees = popular_tutor_rating_df['MenteeID'].nunique()
# num_mentors = popular_tutor_rating_df['TutorID'].nunique()
# # Define embedding dimensions
# embedding_dim_mentees = 32
# embedding_dim_mentors = 16

In [ ]:
# # Create input layers
# mentee_input = Input(shape=(1,))
# mentor_input = Input(shape=(1,))

In [ ]:
# # Create embedding layers
# mentee_embedding = Embedding(input_dim=num_mentees, output_dim=embedding_dim_mentees)(mentee_input)
# mentor_embedding = Embedding(input_dim=num_mentors, output_dim=embedding_dim_mentors)(mentor_input)

In [ ]:
# # Flatten the embeddings
# mentee_flattened = Flatten()(mentee_embedding)
# mentor_flattened = Flatten()(mentor_embedding)

In [ ]:
# # Concatenate user and mentor embeddings
# concatenated = Concatenate()([mentee_flattened, mentor_flattened])

In [ ]:
# # Fully connected layers
# fc1 = Dense(64, activation='relu')(concatenated)
# fc2 = Dense(32, activation='relu')(fc1)
# output = Dense(1)(fc2)

In [ ]:
# # Create the model
# model = Model(inputs=[mentee_input, mentor_input], outputs=output)
# model.compile(optimizer=Adam(0.0001), loss='mean_squared_error', metrics=[RootMeanSquaredError()])

In [ ]:
# # Train the model
# model.fit(x=[train_data['MenteeID'], train_data['TutorID']], y=train_data['Rating'], epochs=150, batch_size=32)
# # Evaluate the model
# test_loss = model.evaluate(x=[test_data['MenteeID'], test_data['TutorID']], y=test_data['Rating'])
# print(f'Test loss: {test_loss}')

#Make Recommendations

In [ ]:
def collaborative_filtering_cosine_similarity(user_id, similarity_matrix, mentors, subject, top_k=5):
    user_idx = mentee_encoder.transform([user_id])[0]
    user_similarity = similarity_matrix[user_idx]
    most_similar_users = np.argsort(user_similarity)[::-1][:top_k]
    recommended_mentors_cs = mentors[mentors['MenteeID'].isin(most_similar_users)]
    # Filter mentors based on subject
    recommended_mentors_cs = recommended_mentors_cs[recommended_mentors_cs['Subjects'] == subject]
    recommended_mentors_cs['TutorID'] = mentor_encoder.inverse_transform(recommended_mentors_cs['TutorID'])
    recommended_mentors_cs['MenteeID'] = mentee_encoder.inverse_transform(recommended_mentors_cs['MenteeID'])
    return most_similar_users

In [ ]:
top_similar_users = collaborative_filtering_cosine_similarity('B55212', rating_similarity, data_cosine, 'Biologi', top_k=5)
top_similar_users

In [ ]:
def data_predict(mentee_id):
  # Convert mentee ID to its encoded index
  mentee_encoded = mentee_encoder.transform([mentee_id])[0]

  # Get mentors that the mentee has not rated
  global mentors_not_rated 
  mentors_not_rated = popular_tutor_rating_df[popular_tutor_rating_df['MenteeID'] != mentee_encoded]['TutorID'].unique()

  # Repeat the mentee ID for all mentors not rated
  mentee_input = np.full(len(mentors_not_rated), mentee_encoded)

  # Reshape the inputs for prediction
  mentee_input = tf.reshape(mentee_input, (len(mentee_input), 1))
  mentor_input = tf.reshape(mentors_not_rated, (len(mentors_not_rated), 1))

  data = [mentee_input, mentor_input]

  return data

In [ ]:
def get_recommendation(data):
  # Get recommendations
  recommendations = model.predict(data).flatten()

  # Sort mentors based on recommendation score
  recommended_mentors = pd.DataFrame({'TutorID': mentors_not_rated, 'Recommendation_score': recommendations})
  recommended_mentors = recommended_mentors.sort_values(by='Recommendation_score', ascending=False)

  ## Decode the output and retrieve original TutorID values
  recommended_mentors['TutorID'] = mentor_encoder.inverse_transform(recommended_mentors['TutorID'])

  # Merge with original mentor data to get subjects
  recommended_mentors = recommended_mentors.merge(mentor_df[['TutorID', 'Subjects']], on='TutorID')
  # Print recommended mentors with original TutorID values and Subjects
  return recommended_mentors.head(5)
# print(recommended_mentors.head(5))

In [ ]:
data_input = data_predict('B55212')

In [ ]:
deep_learning_pred = get_recommendation(data_input)

In [ ]:
# Initialize a weighted prediction array
weighted_predictions = np.zeros_like(deep_learning_pred)
user_id = 'B55212'
# Apply weights to the predictions
for similar_user in top_similar_users:
    similarity_score = cosine_similarity[user_id][similar_user]
    similar_user_predictions = model.predict(similar_user_input)

    weighted_predictions += similarity_score * similar_user_predictions

# Normalize the weighted predictions
weighted_predictions /= np.sum(cosine_similarity[user_id][top_similar_users])

# Combine the predictions from cosine similarity and deep learning
final_predictions = (1 - alpha) * deep_learning_predictions + alpha * weighted_predictions

In [ ]:
collaborative_filtering_cosine_similarity('B55212', rating_similarity, data_cosine, 'Biologi', top_k=5)

In [ ]:
# get_recommendation(data_input, 'B55212', rating_similarity, data_cosine, 'Biologi', top_k=5)

In [ ]:
model.save('Collaborative_Filtering.h5')

In [ ]:
new_model = tf.keras.models.load_model('Collaborative_Filtering.h5')

In [ ]:
new_model.summary()

In [ ]:
# Model 2

# cols_mente = {"ID MENTEE" : "MenteeID", "SUBDISTRICT" : "Subdistrict" ,"NAME" : "MenteeName", "LOCATION" : "Location", "LATITUDE" : "Latitude", "LONGITUDE" : "Longitude"}
# mentee.rename(columns=cols_mente, inplace=True)
# mentee_df = mentee.loc[0:1499, 'MenteeID' : 'Longitude']

# cols_mentor = {"ID MENTOR" :"TutorID",  "SUBDISTRICT" : "Subdistrict", "SUBJECT" : "Subject", "NAME" : "TutorName", "LOCATION" : "Location", "LATITUDE" : "Latitude", "LONGITUDE" : "Longitude"}
# mentor.rename(columns=cols_mentor, inplace=True)
# mentor_df = mentor.loc[0:499, 'TutorID' : 'Longitude']

In [ ]:
# tutor_location = mentor_df.merge(mmentee_df, on="Location")
# tutor_rating.Rating.value_counts()#
#tutor_rating = pd.merge(rating_df, mentor_df, on="TutorID")#
# tutor_location